# Welcome to the Late Fusion Project

Before we start, acknowledgement to this repo: https://github.com/kuixu/kitti_object_vis. This course has been based on this repo after seeing the great results and code! <p>

We'll use the [KITTI Dataset](http://www.cvlibs.net/datasets/kitti/setup.php) to collect the Point Clouds, Images, and Calibration parameters. <p>

After loading data from the dataset, our Late fusion process will happen in 5 steps:
1.   **Detect Obstacles in 2D** with the Camera
2.   **Detect Obstacles in 3D** with the LiDAR
3.   **Project the 3D Obstacles** in the Image (3D Bounding Boxes)
4.   **Fuse the 3D Bounding Box (LiDAR) with the 2D Bounding Box (Camera)**
5.   **Create an ultimate Fused Objects** and Show the results

Are you ready? ✌🏼


##0 - Load the Data and Visualize it!

### Link Google Colab to Google Drive

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

os.chdir("/content/drive/My Drive/Think Autonomous/SDC Course/Visual Fusion")
!ls

### Import the necessary libraries

In [ ]:
!pip install open3d==0.12.0 # Version 12

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import Image
import glob
import open3d as o3d
import pandas as pd

### Load the Files




In [ ]:
image_files = sorted(glob.glob("data/img/*.png"))
point_files = sorted(glob.glob("data/velodyne/*.pcd"))
label_files = sorted(glob.glob("data/label/*.txt"))
calib_files = sorted(glob.glob("data/calib/*.txt"))

print("There are",len(image_files),"images")
index = 0
pcd_file = point_files[index]
image = cv2.cvtColor(cv2.imread(image_files[index]), cv2.COLOR_BGR2RGB)
cloud = o3d.io.read_point_cloud(pcd_file)
points= np.asarray(cloud.points)

### Visualize the Image

In [ ]:
f, (ax1) = plt.subplots(1, 1, figsize=(20,10))
ax1.imshow(image)
ax1.set_title('Image', fontsize=30)

### Visualize the Point Clouds

In [ ]:
!pip install pypotree #https://github.com/centreborelli/pypotree

In [ ]:
import pypotree 
cloudpath = pypotree.generate_cloud_for_display(points)
pypotree.display_cloud_colab(cloudpath)

## 1 - Detect Obstacles in 2D with the Camera
Here's what we should get from YOLO:
* The **2D Bounding Box** coordinates (X1, Y1, X2, Y2)
* The **class** of the object (Car, Pedestrian, ...)
* The **confidence**


### 1.1. Copy/Paste the function to use YOLO

In [ ]:
!python3 -m pip install yolov4==2.0.2 # After Checking, YOLO 2.0.2 works without modifying anything. Otherwise keep 1.2.1

In [ ]:
from yolov4.tf import YOLOv4
import tensorflow as tf
import time

yolo = YOLOv4(tiny=True)
yolo.classes = "Yolov4/coco.names"
yolo.make_model()
yolo.load_weights("Yolov4/yolov4-tiny.weights", weights_type="yolo")

def run_obstacle_detection(img):
    start_time=time.time()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    resized_image = yolo.resize_image(img)
    # 0 ~ 255 to 0.0 ~ 1.0
    resized_image = resized_image / 255.
    #input_data == Dim(1, input_size, input_size, channels)
    input_data = resized_image[np.newaxis, ...].astype(np.float32)

    candidates = yolo.model.predict(input_data)

    _candidates = []
    result = img.copy()
    for candidate in candidates:
        batch_size = candidate.shape[0]
        grid_size = candidate.shape[1]
        _candidates.append(tf.reshape(candidate, shape=(1, grid_size * grid_size * 3, -1)))
        #candidates == Dim(batch, candidates, (bbox))
        candidates = np.concatenate(_candidates, axis=1)
        #pred_bboxes == Dim(candidates, (x, y, w, h, class_id, prob))
        pred_bboxes = yolo.candidates_to_pred_bboxes(candidates[0], iou_threshold=0.35, score_threshold=0.40)
        pred_bboxes = pred_bboxes[~(pred_bboxes==0).all(1)] #https://stackoverflow.com/questions/35673095/python-how-to-eliminate-all-the-zero-rows-from-a-matrix-in-numpy?lq=1
        pred_bboxes = yolo.fit_pred_bboxes_to_original(pred_bboxes, img.shape)
        exec_time = time.time() - start_time
        #print("time: {:.2f} ms".format(exec_time * 1000))
        result = yolo.draw_bboxes(img, pred_bboxes)
        result = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
    return result, pred_bboxes


In [ ]:
result, pred_bboxes = run_obstacle_detection(image)
fig_camera = plt.figure(figsize=(14, 7))
ax_lidar = fig_camera.subplots()
ax_lidar.imshow(result)
plt.show()

### 1.2 - Put these results into "Object2D" and build a list of camera objects

In [ ]:
class Object2D(object):
    def __init__(self, box2D, w, h):
        self.xmin = int(box2D[0]*w - box2D[2]*w/2) # center_x - width/2
        self.ymin = int(box2D[1]*h - box2D[3]*h/2) # center_y - height/2
        self.xmax = int(box2D[0]*w + box2D[2]*w/2) # center_x + width/2
        self.ymax = int(box2D[1]*h + box2D[3]*h/2) # center_y + height/2
        self.bbox = np.array([self.xmin, self.ymin, self.xmax, self.ymax])
        self.category = int(box2D[4])
        self.confidence = box2D[5]

def fill_2D_obstacles(result, pred_bboxes):
    return [Object2D(box, result.shape[1], result.shape[0]) for box in pred_bboxes]

In [ ]:
list_of_2d_objects = fill_2D_obstacles(result, pred_bboxes)

## 2 - Detect Obstacles in 3D

In this part, we are supposed to detect objects in 3D, and then convert that to the camera frame. I won't cover 3D object detection, as we can't really use it easily as a Black Box like YOLO. I invite you to take my course [Point Clouds Fast Course: Introduction to 3D Perception](https://courses.thinkautonomous.ai/point-clouds) to learn more about the process.

<br>

👉 **To simplify the overall process, we're going to directly work with the XYZ Position and Bounding Box dimensions in the camera frame.** Consider that engineers have:
*   Detected Obstacles in 3D (X,Y,Z)
*   Build a Bounding Box in 3D (W,H,L)
*   Computed the orientation of that Bounding Box (Yaw Angle)

👉 When they are detecting these, it's all in the Velodyne Frame.
Then, we apply a conversion to the camera frame; exactly like when we projected the point clouds from the velodyne frame to the camera frame.


In [ ]:
def read_label(label_filename):
    lines = [line.rstrip() for line in open(label_filename)]
    objects = [Object3d(line) for line in lines if line.split(" ")[0]!="DontCare"]
    return objects

In [ ]:
class Object3d(object):
    """ 3d object label """
    def __init__(self, label_file_line):
        data = label_file_line.split(" ")
        data[1:] = [float(x) for x in data[1:]]

        # extract 3d bounding box information
        self.h = data[8]  # box height
        self.w = data[9]  # box width
        self.l = data[10]  # box length (in meters)
        self.t = (data[11], data[12], data[13])  # location (x,y,z) in camera coord.
        self.ry = data[14]  # yaw angle (around Y-axis in camera coordinates) [-pi..pi]

        self.xmin = 0
        self.xmax = 0
        self.ymin = 0
        self.ymax = 0
        self.bbox2d = np.zeros(shape=(2,2))
        self.bbox3d = np.zeros(shape=(4,2))

In [ ]:
list_of_3d_objects = read_label(label_files[index])
for obj3d in list_of_3d_objects:
    print("Object Lateral Position (X), Height(Y), Distance (Z) :"+ str(obj3d.t))

Again, I invite you to take my course [Point Clouds Fast Course: Introduction to 3D Perception](https://courses.thinkautonomous.ai/point-clouds) to learn more about the 3D Perception process.

## 3 - Project the 3D Obstacle to the Image (3D Bounding Box)

From these values, we need to build a 3D Bounding Box in the Image Frame. We'll then fuse that Bounding Box with the Camera boxes.<p>

Here's what we'll do:

*   **Build a LiDAR2Camera Object**
*   Build a function to **Project a 3D Point to a 2D Image**
*   **Build the 3D Bounding Box and project it in the 2D Image**
*   **Draw the Box** for Visualization



### 3.1 - Create a LiDAR2Camera object and fill it with the P value from the camera calibration

In [ ]:
class LiDAR2Camera(object):
    def __init__(self, calib_file):
        self.P = np.reshape(self.read_calib_file(calib_file)["P2"], [3,4])
    
    def read_calib_file(self, filepath):
        """ Read in a calibration file and parse into a dictionary.
        Ref: https://github.com/utiasSTARS/pykitti/blob/master/pykitti/utils.py
        """
        data = {}
        with open(filepath, "r") as f:
            for line in f.readlines():
                line = line.rstrip()
                if len(line) == 0:
                    continue
                key, value = line.split(":", 1)
                # The only non-float values in these files are dates, which
                # we don't care about anyway
                try:
                    data[key] = np.array([float(x) for x in value.split()])
                except ValueError:
                    pass
        return data

### 3.2 - Create a function to project 3D points in 2D

In Early Fusion, we were doing the conversion from the LiDAR to the camera, and then from the camera to the image. *Here, you already have the first step implemented*; so you only need to deal with the conversion **from the camera to the image**.

1.   Convert to Homogeneous Coordinates
2.   Multiply it with P
3.   Divide it with the last element and convert it back to the cartesian coordinates



In [ ]:
def project_to_image(self, pts_3d):
    """ Project 3d points to image plane.
    """
    # Convert to Homogeneous Coordinates
    n = pts_3d.shape[0]
    pts_3d_extend = np.hstack((pts_3d, np.ones((n, 1))))
    # Multiply with the P Matrix
    pts_2d = np.dot(pts_3d_extend, np.transpose(self.P))  # nx3
    # Convert Back to Cartesian
    pts_2d[:, 0] /= pts_2d[:, 2]
    pts_2d[:, 1] /= pts_2d[:, 2]
    return pts_2d[:, 0:2]

LiDAR2Camera.project_to_image = project_to_image

### 3.3 - Compute the 3D Bounding Box from the Values

Create a function that, given a 3D Object (X,Y,Z, W,H,L, RY), outputs a 3D Bounding Box (in the camera frame)

In [ ]:
def roty(t):
    """ Rotation about the y-axis. """
    c = np.cos(t)
    s = np.sin(t)
    return np.array([[c, 0, s], [0, 1, 0], [-s, 0, c]])

def compute_box_3d(self,obj):
        """ Projects the 3d bounding box into the image plane.
            Returns:
                corners_2d: (8,2) array in left image coord.
                corners_3d: (8,3) array in in rect camera coord.
        """
        # compute rotational matrix around yaw axis
        R = roty(obj.ry)
        # 3d bounding box dimensions
        l = obj.l
        w = obj.w
        h = obj.h

        # 3d bounding box corners
        x_corners = [l / 2, l / 2, -l / 2, -l / 2, l / 2, l / 2, -l / 2, -l / 2]
        y_corners = [0, 0, 0, 0, -h, -h, -h, -h]
        z_corners = [w / 2, -w / 2, -w / 2, w / 2, w / 2, -w / 2, -w / 2, w / 2]

        # rotate and translate 3d bounding box
        #corners_3d = np.vstack([x_corners, y_corners, z_corners])
        corners_3d = np.dot(R, np.vstack([x_corners, y_corners, z_corners]))
        corners_3d[0, :] = corners_3d[0, :] + obj.t[0]
        corners_3d[1, :] = corners_3d[1, :] + obj.t[1]
        corners_3d[2, :] = corners_3d[2, :] + obj.t[2]

        # only draw 3d bounding box for objs in front of the camera
        if np.any(corners_3d[2, :] < 0.1):
            corners_2d = None
            return corners_2d

        # project the 3d bounding box into the image plane
        corners_2d = self.project_to_image(np.transpose(corners_3d))
        
        return corners_2d

LiDAR2Camera.compute_box_3d = compute_box_3d

### 3.4 - Create a function to draw a bounding box in 3D, and another one to draw that same bounding box in 2D

In [ ]:
def draw_projected_box3d(self,image, qs, color=(255, 0, 0), thickness=2):
        """ Draw 3d bounding box in image
            qs: (8,3) array of vertices for the 3d box in following order:
                1 -------- 0
            /|         /|
            2 -------- 3 .
            | |        | |
            . 5 -------- 4
            |/         |/
            6 -------- 7
        """
        qs = qs.astype(np.int32)
        for k in range(0, 4):
            # Ref: http://docs.enthought.com/mayavi/mayavi/auto/mlab_helper_functions.html
            i, j = k, (k + 1) % 4
            # use LINE_AA for opencv3
            # cv2.line(image, (qs[i,0],qs[i,1]), (qs[j,0],qs[j,1]), color, thickness, cv2.CV_AA)
            cv2.line(image, (qs[i, 0], qs[i, 1]), (qs[j, 0], qs[j, 1]), color, thickness)
            i, j = k + 4, (k + 1) % 4 + 4
            cv2.line(image, (qs[i, 0], qs[i, 1]), (qs[j, 0], qs[j, 1]), color, thickness)
            i, j = k, k + 4
            cv2.line(image, (qs[i, 0], qs[i, 1]), (qs[j, 0], qs[j, 1]), color, thickness)
        return image

LiDAR2Camera.draw_projected_box3d = draw_projected_box3d

In [ ]:
def project_8p_to_4p(self, pts_2d):
    x0 = np.min(pts_2d[:, 0])
    x1 = np.max(pts_2d[:, 0])
    y0 = np.min(pts_2d[:, 1])
    y1 = np.max(pts_2d[:, 1])
    x0 = max(0, x0)
    # x1 = min(x1, proj.image_width)
    y0 = max(0, y0)
    # y1 = min(y1, proj.image_height)
    return np.array([x0, y0, x1, y1])

LiDAR2Camera.project_8p_to_4p = project_8p_to_4p

In [ ]:
def draw_projected_box2d(self, image, qs, color=(255,0,0), thickness=2):
    return cv2.rectangle(image, (int(qs[0]), int(qs[1])), (int(qs[2]), int(qs[3])), (255,0,0),2)

LiDAR2Camera.draw_projected_box2d = draw_projected_box2d

### 3.5 - Apply the full process and Draw the Box for Visualization

In [ ]:
def get_image_with_bboxes(self,img, objects):
    img2 = np.copy(img)
    img3 = np.copy(img)
    for obj in objects:
        boxes = self.compute_box_3d(obj)
        if boxes is not None:
            obj.bbox3d = boxes
            obj.bbox2d = self.project_8p_to_4p(boxes)
            img2 = self.draw_projected_box2d(img2, obj.bbox2d) # Draw the 2D Bounding Box
            img3 = self.draw_projected_box3d(img3, obj.bbox3d) # Draw the 3D Bounding Box
    return img2, img3

LiDAR2Camera.get_image_with_bboxes=get_image_with_bboxes

In [ ]:
lidar2cam = LiDAR2Camera(calib_files[index])
lidar_2d, lidar_3d =lidar2cam.get_image_with_bboxes(image, list_of_3d_objects)

f,(ax1, ax2)= plt.subplots(1,2, figsize=(20,10))
ax1.imshow(lidar_2d)
ax1.set_title('Image with 2D Bounding Boxes from LiDAR', fontsize=15)
ax2.imshow(lidar_3d)
ax2.set_title('Image with 3D Bounding Boxes from LiDAR', fontsize=15)


## 4 - Fuse the 3D Bounding Box (LiDAR) with the 2D Bounding Box (Camera)
In that step, we have many possibilities, here's what we'll do:
1.   Show both the LiDAR and the Camera boxes on the same images
2.   Compute the IOU Metrics for each box
3.   Send the IOU Matrix to the Hungarian Algorithm that outputs the matches


### 4.1 - Show both lidar and camera boxes

In [ ]:
res = yolo.draw_bboxes(lidar_2d, pred_bboxes)

fig_fusion = plt.figure(figsize=(14, 7))
ax_fusion = fig_fusion.subplots()
ax_fusion.imshow(res)
plt.show()

### 4.2 - Compute the IOU Metrics for every box

In [ ]:
def box_iou(box1, box2):
    """
    Computer Intersection Over Union cost
    """
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])

    inter_area = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # Calculate the Union area by using Formula: Union(A,B) = A + B - Inter(A,B)
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1) #abs((box1[3] - box1[1])*(box1[2]- box1[0]))
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1) #abs((box2[3] - box2[1])*(box2[2]- box2[0]))
    union_area = (box1_area + box2_area) - inter_area

    # compute the IoU
    iou = inter_area/float(union_area)
    return iou

### 4.3 - Send the IOU Matrix to the Hungarian Algorithm that outputs the mathes

In [ ]:
from scipy.optimize import linear_sum_assignment

def associate(lidar_boxes, camera_boxes):
    """
    LiDAR boxes will represent the red bounding boxes
    Camera will represent the other bounding boxes
    Function goal: Define a Hungarian Matrix with IOU as a metric and return, for each box, an id
    """
    # Define a new IOU Matrix nxm with old and new boxes
    iou_matrix = np.zeros((len(lidar_boxes),len(camera_boxes)),dtype=np.float32)

    # Go through boxes and store the IOU value for each box 
    # You can also use the more challenging cost but still use IOU as a reference for convenience (use as a filter only)
    for i,lidar_box in enumerate(lidar_boxes):
        for j,camera_box in enumerate(camera_boxes):
            iou_matrix[i][j] = box_iou(lidar_box, camera_box)

    # Call for the Hungarian Algorithm
    hungarian_row, hungarian_col = linear_sum_assignment(-iou_matrix)
    hungarian_matrix = np.array(list(zip(hungarian_row, hungarian_col)))

    # Create new unmatched lists for old and new boxes
    matches, unmatched_camera_boxes, unmatched_lidar_boxes = [], [], []

    # Go through the Hungarian Matrix, if matched element has IOU < threshold (0.3), add it to the unmatched 
    # Else: add the match    
    for h in hungarian_matrix:
        if(iou_matrix[h[0],h[1]]>0.4):
            matches.append(h.reshape(1,2))
    
    if(len(matches)==0):
        matches = np.empty((0,2),dtype=int)
    else:
        matches = np.concatenate(matches,axis=0)
    
    return matches

In [ ]:
lidar_boxes = [obs.bbox2d for obs in list_of_3d_objects] # Simply get the boxes
camera_boxes = [obs.bbox for obs in list_of_2d_objects]
matches = associate(lidar_boxes, camera_boxes)

print(matches)

## 5 - Create an ultimate Fused Object with all the values

In [ ]:
class FusedObject(object):
    def __init__(self, bbox2d, bbox3d, category, t, confidence):
        self.bbox2d = bbox2d
        self.bbox3d = bbox3d
        self.category = category
        self.t = t
        with open("Yolov4/classes.txt",'rt') as f:
            classes = f.read().rstrip('\n').split('\n')
        self.class_ = classes[category] 

In [ ]:
def build_fused_object(list_of_2d_objects, list_of_3d_objects, matches, image):
    "Input: Image with 3D Boxes already drawn"
    final_image = image.copy()
    list_of_fused_objects = []
    for match in matches:
        fused_object = FusedObject(list_of_2d_objects[match[1]].bbox, list_of_3d_objects[match[0]].bbox3d, 
                                   list_of_2d_objects[match[1]].category,list_of_3d_objects[match[0]].t,
                                   list_of_2d_objects[match[1]].confidence)
        cv2.putText(final_image, '{0:.2f} m'.format(fused_object.t[2]), (int(fused_object.bbox2d[0]),int(fused_object.bbox2d[1])), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (200, 200, 100), 3, cv2.LINE_AA)    
        cv2.putText(final_image, fused_object.class_, (int(fused_object.bbox2d[0]+30),int(fused_object.bbox2d[1]+30)), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (200, 200, 100), 3, cv2.LINE_AA)    
    return final_image, list_of_fused_objects


In [ ]:
final_image, _ = build_fused_object(list_of_2d_objects, list_of_3d_objects, matches, lidar_2d)

plt.figure(figsize=(14,7))
plt.imshow(final_image)
plt.show()

## Create the Complete Pipeline

In [ ]:
def pipeline(img, calib_file):
    lidar2cam = LiDAR2Camera(calib_file)
    # 1 - Run Obstacle Detection
    result, pred_bboxes = run_obstacle_detection(image)

    # 2 - Build a 2D Object
    list_of_2d_objects = fill_2D_obstacles(result, pred_bboxes)
    
    # 3 - Build a 3D Object (from labels)
    list_of_3d_objects = read_label(label_files[index])
    
    # 4 - Get the LiDAR Boxes in the Image in 2D and 3D
    lidar_2d, lidar_3d =lidar2cam.get_image_with_bboxes(image, list_of_3d_objects)
    
    # 5 - Associate the LiDAR boxes and the Camera Boxes
    lidar_boxes = [obs.bbox2d for obs in list_of_3d_objects] # Simply get the boxes
    camera_boxes = [obs.bbox for obs in list_of_2d_objects]
    matches = associate(lidar_boxes, camera_boxes)
    
    #6 - Build a Fused Object
    final_image, _ = build_fused_object(list_of_2d_objects, list_of_3d_objects, matches, lidar_2d)
    
    return final_image


In [ ]:
image_files = sorted(glob.glob("data/img/*.png"))
label_files = sorted(glob.glob("data/label/*.txt"))
calib_files = sorted(glob.glob("data/calib/*.txt"))

print("There are",len(image_files),"images")
index = 1
image = cv2.cvtColor(cv2.imread(image_files[index]), cv2.COLOR_BGR2RGB)
plt.figure(figsize=(14,7))
plt.imshow(pipeline(image, calib_files[index]))
plt.show()